In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

import os
import sys

from cardano_ticker.utils.currency import PriceCurrency
from cardano_ticker.widgets.w_coin_price import BtcPriceWithLogo, EthPriceWithLogo, AdaPriceWithLogo
from cardano_ticker.data_fetcher.data_fetcher import DataFetcher

In [ ]:


data_fetcher = DataFetcher()


In [ ]:
ROOT_PATH = os.path.abspath('../')
ROOT_PATH


In [ ]:
import json
from cardano_ticker.dashboards.dashboard_generator import DashboardGenerator


config_file = os.path.abspath(f'{ROOT_PATH}/src/cardano_ticker/data/config.json')
config = json.load(open(config_file, 'r'))
data_fetcher = DataFetcher(blockfrost_project_id=config['blockfrost_project_id'])
g_dashboard = DashboardGenerator(data_fetcher)





In [ ]:
name, ticker = data_fetcher.pool_name_and_ticker(config["pool_id"])

value_dict = {
    "pool_name": f" [{ticker}] {name} ",
    "pool_id": config["pool_id"],
}
pool_dashboard_path = os.path.abspath(f'{ROOT_PATH}/src/cardano_ticker/dashboards/samples/pool_dashboard_example.json')
json_data = json.load(open(pool_dashboard_path, 'r'))

# update dashboard placeholders
json_data = g_dashboard.update_dashboard_description(json_data, value_dict)


pool_dashboard = g_dashboard.json_to_layout(json_data)

pool_dashboard.render()

In [ ]:



# Load the dashboard json file
price_dashboard_path = os.path.abspath(f'{ROOT_PATH}/src/cardano_ticker/dashboards/samples/price_dashboard_example.json')
json_data = json.load(open(price_dashboard_path, 'r'))
price_dashboard = g_dashboard.json_to_layout(json_data)
price_dashboard.render()

In [ ]:
network_dashboard_path = os.path.abspath(f'{ROOT_PATH}/src/cardano_ticker/dashboards/samples/network_dashboard_example.json')
json_data = json.load(open(network_dashboard_path, 'r'))
network_dashboard = g_dashboard.json_to_layout(json_data)
network_dashboard.render()

In [ ]:



currencies = [PriceCurrency.USD, PriceCurrency.ETH, PriceCurrency.ADA]

btc_price = BtcPriceWithLogo(data_fetcher, (400,100), currencies, background_color=(0.3,0.3,0.3))
btc_price.update()
btc_price.get()

In [ ]:
currencies = [PriceCurrency.USD, PriceCurrency.BTC, PriceCurrency.ETH]

ada_price = AdaPriceWithLogo(data_fetcher, (400,100), currencies, background_color=(0.3,0.3,0.3))
ada_price.update()
ada_price.get()

In [ ]:
currencies = [PriceCurrency.USD, PriceCurrency.EUR]

eth_price = EthPriceWithLogo(data_fetcher, (400,100), currencies, background_color=(0.3,0.3,0.3))
eth_price.update()
eth_price.get()

In [ ]:
from cardano_ticker.widgets.w_plot_chart import PlotChart
btc_usd_chart = PlotChart(data_fetcher, (400,400), 'BTC', 'USD', background_color=(0.7, 0.7, 0.7))
btc_usd_chart.update()
btc_usd_chart.get()

In [ ]:

from cardano_ticker.widgets.generic.w_text import MutiTextWidget
from PIL import ImageFont

# Load fonts
f_file = "/Users/andreigeorgescu/Documents/Develop/eframe/eframe_paper/resources/DejaVuSans.ttf"
font_title = ImageFont.truetype(f_file, 40)


title_widget = MutiTextWidget((510, 80), [("Cardano Pool Dashboard", font_title, "white")], background_color="blue")
title_widget.get()

In [ ]:

data_to_display = {
    "BTC": {
        "USD": 1000,
        "ETH": 0.5
    },
    "ADA": {
        "EUR": 0.8,
        "ETH": 0.0005
    }
}
text_parts = []
for currency, values in data_to_display.items():
    text_parts.append((f"{currency}", font_title, "orange"))
    for currency2, value in values.items():
        text_parts.append((f"\n   {currency2}:", font_title, "white"))
        text_parts.append((f" {value}", font_title, "purple"))
    text_parts.append(("\n", font_title, "white"))

multi_text_widget2 = MutiTextWidget((510, 300), text_parts, background_color="gray")
multi_text_widget2.get()

In [ ]:
from cardano_ticker.widgets.generic.w_table_generic import TableWidget

# Sample data
data = {
    'supply': {
        'max': '45000000000000000',
        'total': '37516667945629674',
        'circulating': '36183827880829453',
        'locked': '393347142320576',
        'treasury': '1646135439887110',
        'reserves': '7483332054370326'
    },
    'stake': {
        'live': '21827731369072329',
        'active': '21783994147298654'
    }
}

# Create table widget for delegators and owners
font_body = ImageFont.truetype(f_file, 20)


# Create table widget for Live Stake and Active Stake
headers = ["Live Stake", "Active Stake"]
rows = [[f"{int(data['stake']['live']) / 1e6} {PriceCurrency.ADA.get_symbol()}", f"{int(data['stake']['active']) / 1e6} {PriceCurrency.ADA.get_symbol()}"]]   
stake_info_widget = TableWidget((600, 100), headers, rows, font_body, header_orientation="columns",  header_color="black", row_color="blue", background_color="grey")

stake_info_widget.get()

In [ ]:
from cardano_ticker.widgets.generic.w_charts_generic import  PieChartWidget


# Create bar widget for supply
chart_data  = [ 
                ("Circulating", int(data['supply']['circulating'])),
                ("Locked", int(data['supply']['locked'])),
                ("Treasury", int(data['supply']['treasury'])), 
                ("Reserves", int(data['supply']['reserves']))
                  ]
bar_chart_colors = [ "green", "blue", "purple", "orange", "yellow"]

# Create new widgets
chart_widget = PieChartWidget((800, 400), chart_data, bar_chart_colors, background_color="white")
chart_widget.get()



class SupplyPieChartWidget(PieChartWidget):
    def __init__(self, data_fetcher, size, background_color=(0.3, 0.3, 0.3), font_size=20):
        super().__init__(size, [], [], background_color, font_size=font_size)
        self.data_fetcher = data_fetcher
        
    def update(self):
        data = self.data_fetcher.network()
        # Create bar widget for supply
        chart_data  = [ 
                        ("Circulating", int(data['supply']['circulating'])),
                        ("Locked", int(data['supply']['locked'])),
                        ("Treasury", int(data['supply']['treasury'])), 
                        ("Reserves", int(data['supply']['reserves']))
                        ]
        bar_chart_colors = [ "green", "blue", "purple", "orange"]
        super().update(chart_data, bar_chart_colors)


supply_pie_chart_widget = SupplyPieChartWidget(data_fetcher, (800, 400), background_color=(0.3, 0.3, 0.3),font_size=14)
supply_pie_chart_widget.update()
supply_pie_chart_widget.get()



In [ ]:
from cardano_ticker.widgets.generic.w_charts_generic import BarChartWidget

# Sample data
data = {
    'supply': {
        'max': '45000000000000000',
        'total': '37516667945629674',
        'circulating': '36183827880829453',
        'locked': '393347142320576',
        'treasury': '1646135439887110',
        'reserves': '7483332054370326'
    },
    'stake': {
        'live': '21827731369072329',
        'active': '21783994147298654'
    }
}
# Sample data for new widgets
chart_data  = [("Live Stake", int(data['stake']['live'])), ("Active Stake", int(data['stake']['active']))]
pie_chart_colors = ["blue", "green"]

# Create new widgets
pie_chart_widget = BarChartWidget((800, 400), chart_data, pie_chart_colors)
pie_chart_widget.get()

pie_chart_widget.get()

In [ ]:

pool_info_table = TableWidget((550, 100), headers, rows, font_body, header_orientation="rows",  header_color="black", row_color="blue", background_color="grey")
pool_info_table.get()


In [ ]:
pool_info_table = TableWidget((550, 100), headers, rows, font_body, header_orientation="columns",  header_color="black", row_color="blue", background_color="grey")
pool_info_table.get()


In [ ]:
from cardano_ticker.widgets.w_pool_history import AdaPoolHistWidget
pool_id = "pool1sj3gnahsms73uxxu43rgwczdw596en7dtsfcqf6297vzgcedquv"
ada_pool_widget = AdaPoolHistWidget(data_fetcher, (600,400), pool_id, background_color=(0.3,0.3,0.3))
ada_pool_widget.render()
ada_pool_widget.get()

In [ ]:
from cardano_ticker.widgets.w_blockchain_stats import BlockchainProgressWidget, BlockchainStatsTable, BlockchainTransactionsWidget


blockchain_progress_widget = BlockchainProgressWidget(data_fetcher, (100, 40), background_color=(0.3, 0.3, 0.3))
blockchain_progress_widget.update()
blockchain_progress_widget.get()

In [ ]:


blockchain_stats_table = BlockchainStatsTable(data_fetcher, (650, 100), background_color="gray", header_orientation="columns")
blockchain_stats_table.update()
blockchain_stats_table.get()



In [ ]:
from cardano_ticker.widgets.w_blockchain_stats import BlockchainTransactionsWidget

blockchain_transactions_widget = BlockchainTransactionsWidget(data_fetcher, (600, 400), background_color="lightgray", font_size=13)
blockchain_transactions_widget.update()
blockchain_transactions_widget.get()